In [2]:
import subprocess as sp

# Print the return code (status=0 mean ping success)
status = sp.call(['tcping', '-c', '1', '-t', '1', 'github.com'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
print(status)

# OR print the full message
status = sp.run(['tcping', '-c', '1', '-t', '1', 'github.com'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
print(status)

0
CompletedProcess(args=['tcping', '-c', '1', '-t', '1', 'github.com'], returncode=0)


In [1]:
from tcping import Ping

def ping_check():
    ping = Ping('18.162.184.7', 80, 1)
    ping.ping(3)

    ret = ping.result.rows
    for r in ret:
        print(r)

    ret = ping.result.raw
    print(ret)

    ret = ping.result.table
    print(ret)

ping_check()

Connected to 18.162.184.7[:80]: seq=1 time=102.65 ms
Connected to 18.162.184.7[:80]: seq=2 time=99.75 ms
Connected to 18.162.184.7[:80]: seq=3 time=105.04 ms
Statistics(host='18.162.184.7', port=80, successed=3, failed=0, success_rate='100.00%', minimum='99.75ms', maximum='105.04ms', average='102.48ms')

--- 18.162.184.7[:80] tcping statistics ---
3 connections, 3 successed, 0 failed, 100.00% success rate
minimum = 99.75ms, maximum = 105.04ms, average = 102.48ms

+--------------+------+-----------+--------+--------------+---------+----------+----------+
|     Host     | Port | Successed | Failed | Success Rate | Minimum | Maximum  | Average  |
+--------------+------+-----------+--------+--------------+---------+----------+----------+
| 18.162.184.7 |  80  |     3     |   0    |   100.00%    | 99.75ms | 105.04ms | 102.48ms |
+--------------+------+-----------+--------+--------------+---------+----------+----------+


In [3]:
a = '100.00%'
b = '105.30ms'

a = float(a.rstrip('%'))
b = float(b.rstrip('ms'))

print(a,b)


100.0 105.3


In [10]:
ip_res = [{'ip': '104.16.86.20', 'delay': 198.99}, {'ip': '151.101.2.109', 'delay': 91.05}, {'ip': '104.16.89.20', 'delay': 161.19}, {'ip': '104.16.85.20', 'delay': 170.16}, {'ip': '104.16.88.20', 'delay': 159.9}, {'ip': '151.101.193.229', 'delay': 90.48}, {'ip': '104.16.87.20', 'delay': 160.54}, {'ip': '151.101.129.229', 'delay': 89.18}]

ip_res.sort(key=lambda x:x['delay'], reverse=False)
print(ip_res)

tmp_re = ip_res[0]
tmp_re.update({'check':1})
ip_res[0] = tmp_re
print(ip_res)




[{'ip': '151.101.129.229', 'delay': 89.18}, {'ip': '151.101.193.229', 'delay': 90.48}, {'ip': '151.101.2.109', 'delay': 91.05}, {'ip': '104.16.88.20', 'delay': 159.9}, {'ip': '104.16.87.20', 'delay': 160.54}, {'ip': '104.16.89.20', 'delay': 161.19}, {'ip': '104.16.85.20', 'delay': 170.16}, {'ip': '104.16.86.20', 'delay': 198.99}]
[{'ip': '151.101.129.229', 'delay': 89.18, 'check': 1}, {'ip': '151.101.193.229', 'delay': 90.48}, {'ip': '151.101.2.109', 'delay': 91.05}, {'ip': '104.16.88.20', 'delay': 159.9}, {'ip': '104.16.87.20', 'delay': 160.54}, {'ip': '104.16.89.20', 'delay': 161.19}, {'ip': '104.16.85.20', 'delay': 170.16}, {'ip': '104.16.86.20', 'delay': 198.99}]
